# 10.1 応用
・量子シミュレーションと化学領域\
・線形代数のスピードアップ\
・最適化\
・テンソルネットワーク\
など

# 10.3 将来展望
量子エラー訂正は量子コンピューターの研究の中でも労力が割かれている部分である。
Surface_code, McCleanによるsubspace expansionによるエラー緩和（VQEの章）、アンチドジッター理論/共形場理論対応から派生したエラー訂正理論などが言及されている。アンチドジッター理論/共形場理論対応は宇宙論と素粒子論・量子論との対応である。その意味で、量子コンピュータへの直接的な応用がなくても物理の研究として活用できる場合もある。

# 10.2 量子超越性
量子超越性とは、最新の古典コンピュータでも効率的に解けない問題を量子コンピュータが効率よく解けることを指す。
ただし、その問題が実用的かどうかは問わない。
量子超越性を示す実験は、（ベルの不等式の破れのように）これまで一般に受け入れられた仮説を覆すことがある。
その一つとして、全てのアルゴリズムは確率的チューリングマシンで効率的にシミュレートできるという仮説(Extended Church-Turing Thesis)が、量子エラー訂正の実験によって破られた例がある。（４章で議論）

## ランダム回路でのサンプリング問題
量子超越性を考える一つの単純な例として、古典的なメモリの表現力がある。
nキュービットの量子系は$2^n$の係数を持ち、その振幅を表すのに8バイト必要、などを考えると現状のRAMなどで扱えないキュービット数などを計算することはできる。
キュービット数が増えると、古典的枠組みにおいて（物理的もしくは時間的に）リソースが指数関数的に増える、というのがランダム回路の量子超越性においての根本的なアイデアである。
以下の論文の内容に沿って量子超越性について議論する。
"Characterizing Quantum Supremacy in Near-Term Devices" https://arxiv.org/pdf/1608.00263.pdf

まず、ランダム回路を以下のように定義する。\
※ランダム回路とは、出力パターンが均一にランダムになるものではなく、ある出力パターン{$x_j$}$_{j=1..N}$（$N=2^n$）が実現する確率が$p$のとき、同じ$p$の確率で出現する出力パターンの割合が$e^{-Np}$のように分布する量子回路のことを言う。
以下の回路はその性質を満たす。
1. アダマールゲートを全キュービットに作用
2. 隣接するキュービットについて、Cnotrolled_Zゲートを気まぐれに作用させる。（作用させなくてもよい）
3. 以下の基準で{$X^{1/2}$,$Y^{1/2}$,$T$}を作用させる。($T=[[1,0], [0, e^{i \pi/4}]]$)\
 ・前のサイクル（回路でいう一つ左）でCZゲートが使われていれば、$X^{1/2}$または$Y^{1/2}$を作用。\
 ・前のサイクルで$X^{1/2}$または$Y^{1/2}$が使われていれば、{$X^{1/2}$,$Y^{1/2}$,$T$}（ただし2.でCZが作用させられていない場合に限る）\
 ・これまでのサイクルで{$X^{1/2}$,$Y^{1/2}$,$T$}が使われれいなければ$T$を作用。\
 ・上のどれも満たされていなければ何もしない。
4. 2.と3.を決められたサイクルだけ繰り返す。
5. Z基底かX基底で測定する

実際に回路を生成するコードは以下のとおり。4×4の二次元格子に並んだキュービットを考える。
cirqの最新バージョンではexperiments.generate_supremacy_circuit_google_v2_gridがサポートされていないためコードだけ載せる。

In [10]:
import cirq

#Number of rows in grid of quibts
nrows = 4

#Number of columns in grid of qubits
ncols = 4

# Depth of CZ gates in supremacy circuit
depth = 5

#Generate  the supremacy circuit
supremacy_circuit = cirq.experiments.generate_supremacy_circuit_google_v2_grid(nrows, ncols, depth, seed=123)

print(supremacy_circuit)

AttributeError: module 'cirq.experiments' has no attribute 'generate_supremacy_circuit_google_v2_grid'

この回路を用いて量子超越性を確かめる実験は以下の通り。
1. キュービット数$n$で深さ$d$のランダム回路$U$を作る。
2. 回路の結果を$m \sim10^3-10^6$回出力する。結果を{$\vec{x}_j$}$_{j=1..m}$とする。
3. $1/p_U (\vec{x}_j)$を古典計算機で求める。ただし$|\psi \rangle = U |{\psi_0}\rangle $を回路の終状態として
\begin{equation}
p_U (\vec{x}_j) := |\langle \vec{x}_j | \psi \rangle | ^2
\end{equation}
4. 以下の値を計算
\begin{equation}
\alpha = H_0 - \frac{1}{m} \sum_{j=1}^{m} {\rm log} \frac{1}{p_U (\vec{x}_j) }
\end{equation}
ここで、$H_0 = {\rm log} (2^n) + \gamma$  ($\gamma \sim 0.577$はオイラー定数)は出力が均一ランダムな回路があったときの交差エントロピー。
5. 後述する$C$を用いて以下が成立していれば量子超越性があると言える。
\begin{equation}
C \leq \alpha  \leq 1
\end{equation}

次に$\alpha$と$C$の意味について考える。$p(\vec{x}_j )$を$\vec{x}_j $に対する任意の確率分布として、$\Delta H(p)$を
\begin{equation}
\Delta H(p) := H_0 - H(p, p_U)
\end{equation}
と定義する。ただし$H(p, p_U)　 =  -\sum p {\rm log} p_U$は交差エントロピーで$p$と$p_U$が近いほど小さくなる。逆に、$\Delta H(p)$は$p$と$p_U$の近さを評価する値となる。これを用いて、$\alpha$と$C$は以下のように表される。

\begin{equation}
\alpha = E_U(\Delta H(p_{exp}) ),~~~C = E_U(\Delta H(p_A) )
\end{equation}

ただし$E_U$は$U$に関するアンサンブル期待値である。$p_{exp}$は実際に量子回路を動かして動かして得られた確率分布で、$\kappa_U$をノイズ含めた回路の写像とすると、$p_{exp}(\vec{x}_j|U) = \langle \vec{x}_j |\kappa_U(| \psi_0 \rangle \langle \psi_0| ) | \vec{x}_j  \rangle$と表される。$p_A$は現存する古典的コンピュータで最良のアルゴリズムを使った場合に得られる出力の確率分布である。つまり、理想的な量子ランダム回路の出力$p_U$に対して、$\alpha$はノイズ含めた量子回路の出力分布がどの程度$p_U$に近いかを表し、$C$は現存の古典コンピュータが最善を尽くすとどの程度$p_U$を予測できるかを表している。これより、$C\leq \alpha$であれば量子超越性がある、と理解できる。

回路のサイズが大きくなれば、古典計算は難しくなり、$C$が0に近づくため量子超越性は見かけ上実現しやすくなる。
しかし、回路のサイズが大きくなればノイズが大きくなり、さらに$p_U$を求めるための統計的なサンプル出力が多く必要になるため、$p_{exp}$から$p_U$を求めるのが難しくなっていく。つまり$\alpha$を求めるのが難しくなっていく。この求め方は論文参照。https://arxiv.org/pdf/1608.00263.pdf

（呟き）ちなみに、$\alpha$が4.の形にかけるのは、$U$に関するアンサンブルを取っているからとのこと。1/mはわかるが、log(1/p_U)の部分はそのままでよいのか？